# IO


## Export Tiff Image with Meta Data

In [ ]:
from scitiff.data import hyperstack_example
import scipp as sc

sample_img = hyperstack_example()
sample_img

In [ ]:
config = {"grid": True, "aspect": True}
img = sample_img.mean('c').mean('z')
(img['t', 0].plot(title='T=0', **config) + img['t', 1].plot(title='T=1', **config)) / (
    img['t', 2].plot(title='T=2', **config) + img['t', 3].plot(title='T=3', **config)
)

In [ ]:
from scitiff.io import save_scitiff

save_scitiff(sample_img, 'sample_img.tiff')

## Load TIFF Image with Meta Data

In [ ]:
from scitiff.io import load_scitiff

loaded_scitiff: sc.DataGroup[sc.DataArray] = load_scitiff('sample_img.tiff')
loaded_scitiff

In [ ]:
loaded_scitiff['image']

In [ ]:
config = {"grid": True, "aspect": True}
loaded_img = loaded_scitiff['image'].mean('c').mean('z')
(
    loaded_img['t', 0].plot(title='T=0', **config)
    + loaded_img['t', 1].plot(title='T=1', **config)
) / (
    loaded_img['t', 2].plot(title='T=2', **config)
    + loaded_img['t', 3].plot(title='T=3', **config)
)

## Show Metadata Only

In [ ]:
from scitiff.executables import show_metadata

show_metadata('sample_img.tiff')

## Load Non-Scitiff Images

Sometimes you might want to load tiff images that are not compatible with scitiff.
i.e. no metadata, incompatible dtype, or broken metadata with wrong fields or values.

`load_scitiff` will try to load metadata and handle incompatible profiles, but if it is not possible it will load the tiff image as it is and wrap it into scipp data structure.

.. note::
    `load_scitiff` will throw warning if it fails to load the file as expected way
    so please pay attention to the warning messages.

In [ ]:
import tifffile as tf

tf.imwrite('no-meta.tiff', data=sample_img.values)
no_meta_loaded_image = load_scitiff('no-meta.tiff')['image']
display(no_meta_loaded_image)
no_meta_loaded_image_reduced = no_meta_loaded_image.sum('dim_0').sum('dim_2')
(
    no_meta_loaded_image_reduced['dim_1', 0].plot(title='dim_1=0', **config)
    + no_meta_loaded_image_reduced['dim_1', 1].plot(title='dim_1=1', **config)
) / (
    no_meta_loaded_image_reduced['dim_1', 2].plot(title='dim_1=2', **config)
    + no_meta_loaded_image_reduced['dim_1', 3].plot(title='dim_1=3', **config)
)

.. tip::
    You can rename the dimensions and use it just like other scitiff images.
    Do it only if you know which `dim_{i}` corresponds to which of ('t', 'z', 'c', 'y', 'x').

In [ ]:
renamed_image = no_meta_loaded_image.rename_dims(
    {"dim_0": "c", "dim_1": "t", "dim_2": "z", "dim_3": "y", "dim_4": "x"}
)
display(renamed_image)
renamed_image_reduced = renamed_image.sum('z').sum('c')
(
    renamed_image_reduced['t', 0].plot(title='T=0', **config)
    + renamed_image_reduced['t', 1].plot(title='T=1', **config)
) / (
    renamed_image_reduced['t', 2].plot(title='T=2', **config)
    + renamed_image_reduced['t', 3].plot(title='T=3', **config)
)

## Mask and Standard Deviation

.. warn:: This is not officially declared in the scitiff-schema.

In some cases, users need to save variances and masks into the file along with the counts.

For example, a simulated image is normalized with a background image with some 0 counts pixels.<br>
Then the variances are not simply same as the counts and 0 counts pixels of background image should be masked.

It is not practical to save such information in the plain text json.

Therefore `scitiff.io` has helpers to save/load `mask` and `variances` as channels.

.. tip:: If masks can be represented as 1 dimensional array, it is better to keep it as masks.


Here is the sample image we want to save and load.<br>
The image stack has 4 dimensions, (t, x, y, z) data, a mask with same dimension and variances.<br>
If we save it as it is, the mask is not accepted as it is not 1 dimensional array<br>
and the variances will be simply ignored.

In [ ]:
from scitiff.data import hyperstack_example_with_variances_and_mask

sample_img_with_stdev_and_mask = hyperstack_example_with_variances_and_mask()
display(sample_img_with_stdev_and_mask)
sample_img_with_stdev_and_mask['t', 0]['z', 0].plot(title="Masked Image, T=0, Z=0") + \
sample_img_with_stdev_and_mask['t', 1]['z', 0].plot(title="Masked Image, T=1, Z=0") + \
sample_img_with_stdev_and_mask['t', 2]['z', 0].plot(title="Masked Image, T=2, Z=0")

### Save Mask and Standard Deviation

In order to save the mask and the variances,

you can simply set `concat_stdevs_and_mask` argument to `True` of `save_scitiff`.

Then it will concatenate stdevs and mask into channel dimension before it saves the image.

In [ ]:
from scitiff.io import save_scitiff

save_scitiff(
    sample_img_with_stdev_and_mask,
    'sample_img_with_stdev_and_mask.tiff',
    concat_stdevs_and_mask=True,
    mask_name='mask',
)

Or you can manually translate the image to have stdevs and mask concatenated into channel dimension.

Note that the `c` (channel) coordinate has `string` values that indicates which channel is `intensities`, `stdevs` or `mask`.

And you can save the object as it is.

In [ ]:
from scitiff.io import save_scitiff
from scitiff.io import to_scitiff_image

da = to_scitiff_image(
    sample_img_with_stdev_and_mask, concat_stdevs_and_mask=True, mask_name='mask'
)
display(da)
save_scitiff(da, 'sample_img_with_stdev_and_mask.tiff')

### Load Mask and Standard Deviation

As the `stdevs` and `mask` is saved into the tiff stack, you can load them as well.

In [ ]:
import scipp as sc
from scitiff.io import load_scitiff

img_with_stdev_and_mask_dg = load_scitiff('sample_img_with_stdev_and_mask.tiff')
img_with_stdev_and_mask = img_with_stdev_and_mask_dg['image']
display(img_with_stdev_and_mask)
tiled = img_with_stdev_and_mask['c', sc.scalar('intensities')]['t', 0]['z', 0].plot(title='Loaded Image (intensities), T=0, Z=0') / \
img_with_stdev_and_mask['c', sc.scalar('stdevs')]['t', 0]['z', 0].plot(title='Loaded Image (stdevs), T=0, Z=0') / \
img_with_stdev_and_mask['c', sc.scalar('mask')]['t', 0]['z', 0].plot(title='Loaded Image (mask), T=0, Z=0')
tiled

There is a helper function that reassemble the data array with mask and variances from the loaded image.

In [ ]:
from scitiff.io import resolve_scitiff_channels

img_with_stdev_and_mask_resolved = resolve_scitiff_channels(img_with_stdev_and_mask)
display(img_with_stdev_and_mask_resolved)
img_with_stdev_and_mask_resolved['t', 0]['z', 0].plot(title='Loaded Image (intensities), T=0, Z=0')

Or you can simply set the `resolve_channels` argument to be `True` and the loader will try reassembling the data array with mask and variances.

In [ ]:
import scipp as sc
from scitiff.io import load_scitiff

img_with_stdev_and_mask_dg = load_scitiff('sample_img_with_stdev_and_mask.tiff', resolve_channels=True)
img_stdev_and_mask_resolved_at_once = img_with_stdev_and_mask_dg['image']

display(img_stdev_and_mask_resolved_at_once)
img_stdev_and_mask_resolved_at_once['t', 0]['z', 0].plot(title='Masked Image, T=0, Z=0')